<i>Written by: Laura Brin, Sandra Alex & Annabell Rodriguez, On behalf of Norquest College Institute for the CMPT-3510 Machine Learning I Fall course</i>

# House Price Regression

In [2]:
#Crucial data processing and analysis libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


# Metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

# loading the house sales csv data
df = pd.read_csv("kc_house_data.csv")

# This makes it so we are able to see 100 rows when displaying the data
pd.set_option("display.max_rows", 100)

### Problem definition

space saver

In [ ]:
#code saver

### Visualizing and Cleaning data- Laura

sapce saver

In [ ]:
#code saver

### Regression

space saver

In [ ]:
#code saver